In [1]:
import pandas as pd
base = pd.read_csv('house_prices.csv')
base.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [2]:
house_sizes = base.iloc[:, 5:6].values
prices = base.iloc[:, 2:3].values

In [3]:
from sklearn.preprocessing import StandardScaler
scaler_house_sizes = StandardScaler()
house_sizes_transformed = scaler_house_sizes.fit_transform(house_sizes)

scaler_prices = StandardScaler()
prices_transformed = scaler_prices.fit_transform(prices)

In [4]:
import tensorflow as tf

In [5]:
columns = [tf.feature_column.numeric_column('previsor', shape = [1])]

In [6]:
regressor = tf.estimator.LinearRegressor(feature_columns = columns, config = tf.estimator
.RunConfig(tf_random_seed = 42))

INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Matheus\\AppData\\Local\\Temp\\tmp0o7e29w1', '_tf_random_seed': 42, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [7]:
from sklearn.model_selection import train_test_split
house_size_train, house_size_test, price_train, price_test = train_test_split(
  house_sizes_transformed, prices_transformed, test_size = 0.3, random_state= 42)

In [8]:
print(house_size_train.shape, house_size_test.shape,
      price_train.shape, price_test.shape)


(15129, 1) (6484, 1) (15129, 1) (6484, 1)


In [9]:
def make_input_fn(house_size, price = None, num_epochs = None, shuffle = True, batch_size = 32):
    def input_function():
        # Precisamos criar um dataset com as variáveis previsoras e de saída. 
        # As variáveis previsoras precisam ser passadas como um dicionário com os mesmos nomes definidos na chamada de `tf.feature_column.numeric_column`
        ds = tf.data.Dataset.from_tensor_slices(({'previsor': house_size}, price))
        if shuffle:
            ds = ds.shuffle(1000)
        ds = ds.batch(batch_size).repeat(num_epochs)
        return ds
    return input_function

In [10]:
train_function = make_input_fn(house_size_train, price_train)
test_function = make_input_fn(house_size_test, price_test, num_epochs = 1000, shuffle = False)

In [11]:
regressor.train(input_fn = train_function, steps = 10000)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


C:\Users\Matheus\.conda\envs\machine_learning\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\Matheus\AppData\Local\Temp\tmp0o7e29w1\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 0.8920282, step = 0
INFO:tensorflow:global_step/sec: 1470.56
INFO:tensorflow:loss = 0.30374563, step = 100 (0.069 sec)
INFO:tensorflow:global_step/sec: 2083.38
INFO:tensorflow:loss = 0.4635738, step = 200 (0.048 sec)
INFO:tensorflow:global_step/sec: 2061.83
INFO:tensorflow:loss = 0.241681, step = 300 (0.048 sec)
INFO:tensorflow:global_step/sec: 2150.5
INFO:tensorflow:loss = 0.37792256, step = 400 (0.047 sec)
INFO:tensorflow:global_step/sec: 2150.55
INFO:tensorflow:loss = 1.0386455, step = 500 (0.047 sec)
INFO:tens

In [12]:
metrics_train = regressor.evaluate(input_fn= train_function, steps = 10000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2022-03-02T20:59:06
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Matheus\AppData\Local\Temp\tmp0o7e29w1\model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1000/10000]
INFO:tensorflow:Evaluation [2000/10000]
INFO:tensorflow:Evaluation [3000/10000]
INFO:tensorflow:Evaluation [4000/10000]
INFO:tensorflow:Evaluation [5000/10000]
INFO:tensorflow:Evaluation [6000/10000]
INFO:tensorflow:Evaluation [7000/10000]
INFO:tensorflow:Evaluation [8000/10000]
INFO:tensorflow:Evaluation [9000/10000]
INFO:tensorflow:Evaluation [10000/10000]
INFO:tensorflow:Inference Time : 3.78540s
INFO:tensorflow:Finished evaluation at 2022-03-02-20:59:09
INFO:tensorflow:Saving dict for global step 10000: average_loss = 0.48810703, global_step = 10000, label/mean = -0.0065834196, loss = 0.488

In [13]:
test_metrics = regressor.evaluate(input_fn = test_function, steps = 10000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2022-03-02T20:59:10
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Matheus\AppData\Local\Temp\tmp0o7e29w1\model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1000/10000]
INFO:tensorflow:Evaluation [2000/10000]
INFO:tensorflow:Evaluation [3000/10000]
INFO:tensorflow:Evaluation [4000/10000]
INFO:tensorflow:Evaluation [5000/10000]
INFO:tensorflow:Evaluation [6000/10000]
INFO:tensorflow:Evaluation [7000/10000]
INFO:tensorflow:Evaluation [8000/10000]
INFO:tensorflow:Evaluation [9000/10000]
INFO:tensorflow:Evaluation [10000/10000]
INFO:tensorflow:Inference Time : 3.56550s
INFO:tensorflow:Finished evaluation at 2022-03-02-20:59:13
INFO:tensorflow:Saving dict for global step 10000: average_loss = 0.55330914, global_step = 10000, label/mean = 0.015570629, loss = 0.55679

In [14]:
print(metrics_train)
print(test_metrics)

{'average_loss': 0.48810703, 'label/mean': -0.0065834196, 'loss': 0.48807675, 'prediction/mean': -0.012411411, 'global_step': 10000}
{'average_loss': 0.55330914, 'label/mean': 0.015570629, 'loss': 0.55679685, 'prediction/mean': 0.0032317184, 'global_step': 10000}


In [21]:
import numpy as np
new_houses_transformed = scaler_house_sizes.transform(np.array([[800], [900], [1000], [200]]))
predict_function = make_input_fn(new_houses_transformed, num_epochs = 1, shuffle = False)
predicts = regressor.predict(input_fn = predict_function)
list(predicts)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Matheus\AppData\Local\Temp\tmp0o7e29w1\model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


C:\Users\Matheus\.conda\envs\machine_learning\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


[{'predictions': array([-0.96557844], dtype=float32)},
 {'predictions': array([-0.8907409], dtype=float32)},
 {'predictions': array([-0.8159032], dtype=float32)},
 {'predictions': array([-1.414604], dtype=float32)}]

In [22]:
predicts_results = []
for predict in regressor.predict(input_fn = predict_function):
    predicts_results.append(predict['predictions'])
predicts_results = np.asarray(predicts_results).reshape(-1, 1)
predicts_results = scaler_prices.inverse_transform(predicts_results)
predicts_results

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Matheus\AppData\Local\Temp\tmp0o7e29w1\model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


array([[185606.23],
       [213080.52],
       [240554.83],
       [ 20760.58]], dtype=float32)